problem1  assign value to tensor variable

In [1]:
import tensorflow as tf
from tensorflow.contrib.eager.python import tfe
import numpy as np

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected


# problem1 :assign value to tensor variable

In [9]:
tf.reset_default_graph()
W=tf.get_variable('W',shape=[])

sess=tf.Session()
sess.run(tf.global_variables_initializer())
sess.run(W.assign(2.0))
# sess.run(tf.global_variables_initializer())
# sess.run(W.initializer)
print(sess.run(W))
sess.close()

-0.6661053


### problem1_1:find all uninitialize variable

In [37]:
tf.reset_default_graph()
sess=tf.Session()
W1=tf.get_variable('W1',shape=[2,3])
W2=tf.get_variable('W2',shape=[3,2])

uninit_vars=tf.report_uninitialized_variables()
print(sess.run(uninit_vars))

sess.run(W1.assign(np.zeros((2,3))))
print(sess.run(uninit_vars))

# sess.run(tf.global_variables_initializer())
need_init_vars=sess.run(uninit_vars)
op=[]
with tf.variable_scope('',reuse=True):
    for v in need_init_vars:
        name=v.decode("utf-8")
        init_op=[v.initializer for v in tf.global_variables(name)]
        op.extend(init_op)
sess.run(op)
print(sess.run(uninit_vars))
print(sess.run(W2))
sess.close()

[b'W1' b'W2']
[b'W2']
[]
[[ 0.45917213  0.11933637]
 [ 0.24807417 -0.84282833]
 [ 0.5821041  -0.30012006]]


# problem2:how to remove trainable variable

In [ ]:
tf.reset_default_graph()
def nn():
    W1=tf.get_variable('W1',shape=[])
    W2=tf.get_variable('W2',shape=[])
    cost=W1+W2
    return (W1,W2,cost)
W1,W2,cost=nn()
opt=tf.train.GradientDescentOptimizer(1).minimize(cost)


sess=tf.InteractiveSession()
sess.run(W1.assign(5.0))
sess.run(W2.assign(5.0))

for i in range(1):
    sess.run(opt)
print(tf.trainable_variables())
print(cost.eval()) #output 8

opt=tf.train.GradientDescentOptimizer(1).minimize(cost,var_list=[W2])
#train nn again
for i in range(1):
    sess.run(opt) 
print(cost.eval()) #output 7

## problem3:share parameter

In [ ]:
tf.reset_default_graph()
tf.set_random_seed(1)

def nn(X):
    #create variables if they do not exist, and return them otherwise;
    with tf.variable_scope('s1',reuse=tf.AUTO_REUSE):  
        W=tf.get_variable('W',shape=[])
        out=X*W
    return out
def compute_cost(f1,f2):
    return f1+f2
X1=tf.constant(-5.0)
X2=tf.constant(3.0)

loss=compute_cost(nn(X1),nn(X2)) #loss=-2W,gradient=-2 new loss=-2(w+2)=-2w-4
opt=tf.train.GradientDescentOptimizer(1).minimize(loss)

sess=tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
for i in range(3):
    sess.run(opt)
    _c=sess.run(loss)
    #notice grad=X1+X2,cost=X1(W-X1-X2)+X2(W-X1-X2)=cost_old-(X1+X2)**2
    print(_c)


# WEEK 1 <a href='https://www.coursera.org/learn/intro-tensorflow/lecture/n691C/tensors'>turtorial 1</a>

In [ ]:
import tensorflow as tf
import tensorboard
import numpy as np
#tensorflow can work on eager mode,notice  tfe.enable_eager_execution() must be call at startup,call only once

In [ ]:
# tfe.enable_eager_execution()
a=tf.constant(1.0)
print(a)

In [ ]:
#evaluate tensor
with tf.Session() as sess:
    print(a.eval()) #eval mean evaluate a at content of sess,a shortcut of sess.run(a)

In [ ]:
#visualization,using tensorboard

x=tf.constant(3.0,name='x')
y=tf.constant(3.0,name='y')
z=tf.add(x,y,name='z')

In [ ]:
with tf.Session() as sess:
    #tensorboard --port 8080 --logdir=/home/zhangxk/PycharmProjects/untitled/deepAI/daily/8/summary
    with tf.summary.FileWriter('summary',sess.graph) as writer:
        pass
    

<img src='images/tensor.png' />

<a href='https://www.tensorflow.org/api_docs/python/tf/stack'>stack doc</a>

In [ ]:
def tensorEx():
    tf.reset_default_graph()
    x1=tf.constant([1,2,3],name='x1') #(3,)
    x2=tf.stack([x1 ,x1],name='x2') #(2,3)
    x3=tf.stack([x2,x2,x2,x2],name='x3') #(4,2,3)
    x4=tf.stack([x3,x3],name='x4') #(2,4,2,3)
    print('x1 shape:'+str(x1.shape))
    print('x2 shape:'+str(x2.shape))
    print('x3 shape:'+str(x3.shape))
    print('x4 shape:'+str(x4.shape))
    
    #this is a example of inception implement
    conv3=tf.constant(np.random.randn(100,14,14,32),name='3x3')
    conv5=tf.constant(np.random.randn(100,14,14,16),name='5x5')
    conv7=tf.constant(np.random.randn(100,14,14,8),name='7x7')
    out=tf.concat([conv3,conv5,conv7],axis=3,name='out')
    print('out shape:'+str(out.shape))
    with tf.Session() as sess:
        with tf.summary.FileWriter('summary/',sess.graph) as writer:
            pass

In [ ]:
tensorEx()

## Common Error
### 1.Shape error
<a href='https://www.tensorflow.org/api_docs/python/tf/slice'>slice</a> <a href='https://www.tensorflow.org/api_docs/python/tf/squeeze'>squeeze</a> <a href='https://www.tensorflow.org/api_docs/python/tf/expand_dims'>expand_dims</a>

In [ ]:
def shape_error():
    A=tf.constant(shape=[2,3,4],value=1.0)
    B=tf.constant(shape=[2,3],value=1.0)
    #to fix error
    
    #1.B need rank 3,so expend [2,3,1] then brost cast
    #B=tf.expand_dims(B,axis=2)
    
    #2.convert A to a (2,3) matrix
    A=tf.slice(A,begin=[0,0,0],size=[2,3,1]) #A[0:2,0:3,0:1],shape[2,3,1]
    A=tf.squeeze(A,axis=-1)
    
    C=A+B
    
    print(C.get_shape())
shape_error()

### 2.DataType errpr <a href='https://www.tensorflow.org/api_docs/python/tf/cast'>cast</a> 

In [ ]:
def dataType_error():
    A=tf.constant(shape=[2,3],value=1)
    B=tf.constant(shape=[2,3],value=1.0)
    
    B=tf.cast(B,tf.int32)
    C=A+B
dataType_error()

### 3. USE debuger

In [ ]:
import tensorflow as tf
from tensorflow.python import debug as tf_debug

def mydiv(a, b):
    s = tf.div(a,b,name='a/b')
    s2 = tf.matmul(s, tf.transpose(s,name='s_transpose'),name='s_normal2')
    return tf.sqrt(s2,name='s_normal')

with tf.Session() as sess:
    fake_a = [
      [5.0, 3.0, 7.1],
      [2.3, 4.1, 4.8],
    ]
    fake_b = [
      [2, 0, 5],
      [2, 8, 7]
    ]
    a = tf.placeholder(tf.float32, shape=[2, 3],name='a')
    b = tf.placeholder(tf.float32, shape=[2, 3],name='b')
    k = mydiv(a, b)
  
  # Note: won't work without the ui_type="readline" argument because
  # Datalab is not an interactive terminal and doesn't support the default "curses" ui_type.
  # If you are running this a standalone program, omit the ui_type parameter and add --debug
  # when invoking the TensorFlow program
  #      --debug (e.g: python debugger.py --debug )
    sess = tf_debug.LocalCLIDebugWrapperSession(sess, ui_type="readline")
    sess.add_tensor_filter("has_inf_or_nan", tf_debug.has_inf_or_nan)
    print(sess.run(k, feed_dict = {a: fake_a, b: fake_b}))

In the tfdbg> window that comes up, try the following:
* run -f has_inf_or_nan
* Notice that several tensors are dumped once the filter criterion is met
* List the inputs to a specific tensor:
* li transpose:0 
* Print the value of a tensor
* pt transpose:0
* Where is the inf?

Visit https://www.tensorflow.org/programmers_guide/debugger for usage details of tfdbg

In [1]:
#Use of tf.train.range_input_producer
import tensorflow as tf
sess=tf.Session()
# m=tf.constant(10,dtype=tf.int32)
m=1200
epochs=None

In [ ]:
range_queue=tf.train.range_input_producer(m,num_epochs=epochs,capacity=32)
print(range_queue)
one_depart=range_queue.dequeue_many(m)
print(one_depart)

In [ ]:
sess.run(one_depart)

In [ ]:
NUM_EXPOCHES=3
i = tf.train.range_input_producer(NUM_EXPOCHES, num_epochs=2, shuffle=False).dequeue()
# inputs = tf.slice(array, [i * BATCH_SIZE], [BATCH_SIZE])
sess.run(i)

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.range(limit).shuffle(limit).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
